# Model Tuning

In [3]:
import matchzoo as mz
train_raw = mz.datasets.toy.load_data('train')
dev_raw = mz.datasets.toy.load_data('dev')
test_raw = mz.datasets.toy.load_data('test')

## basic Usage

A couple things are needed by the tuner:
 - a model with a parameters filled
 - preprocessed training data
 - preprocessed testing data
 
Since MatchZoo models have pre-defined hyper-spaces, the tuner can start tuning right away once you have the data ready.

### prepare the data

In [4]:
preprocessor = mz.models.DenseBaseline.get_default_preprocessor()
train = preprocessor.fit_transform(train_raw, verbose=0)
dev = preprocessor.transform(dev_raw, verbose=0)
test = preprocessor.transform(test_raw, verbose=0)

### prepare the model

In [6]:
model = mz.models.DenseBaseline()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = mz.tasks.Ranking()

### start tuning

In [7]:
results = mz.tune.tune(
    params=model.params,
    train_data=train,
    test_data=dev,
    num_runs=5
)

Run #1
Score: 0.07142857142857142
model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 165
mlp_num_layers                4
mlp_num_fan_out               52
mlp_activation_func           relu

Run #2
Score: 0.0625
model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 186
mlp_num_layers                3
mlp_num_fan_out               84
mlp_activation_func           relu

Run #3
Score: 0.07142857142857142
model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  [(30,), (30,)]
task                     

### view the best hyper-parameter set

In [9]:
results['best']

{'#': 5,
 'params': <matchzoo.engine.param_table.ParamTable at 0x138943d68>,
 'sample': {'mlp_num_fan_out': 60.0,
  'mlp_num_layers': 2.0,
  'mlp_num_units': 455.0},
 'score': 0.1}

In [14]:
results['best']['params'].to_frame()

,Name,Description,Value,Hyper-Space
0,model_class,Model class. Used internally for save/load. Ch...,<class 'matchzoo.models.dense_baseline.DenseBa...,None
1,input_shapes,Dependent on the model and data. Should be set...,"[(30,), (30,)]",None
2,task,"Decides model output shape, loss, and metrics.",Ranking Task,None
3,optimizer,None,adam,None
4,with_multi_layer_perceptron,A flag of whether a multiple layer perceptron ...,True,None
5,mlp_num_units,Number of units in first `mlp_num_layers` layers.,455,"quantitative uniform distribution in [16, 512..."
6,mlp_num_layers,Number of layers of the multiple layer percetron.,2,"quantitative uniform distribution in [1, 5), ..."
7,mlp_num_fan_out,Number of units of the layer that connects the...,60,"quantitative uniform distribution in [4, 128)..."
8,mlp_activation_func,Activation function used in the multiple layer...,relu,None


## understading hyper-space

`model.params.hyper_space` reprensents the model's hyper-parameters search space, which is the cross-product of individual hyper parameter's hyper space. When a `Tuner` builds a model, for each hyper parameter in `model.params`, if the hyper-parameter has a hyper-space, then a sample will be taken in the space. However, if the hyper-parameter does not have a hyper-space, then the default value of the hyper-parameter will be used.

In [15]:
model.params.hyper_space

{'mlp_num_units': <hyperopt.pyll.base.Apply at 0x136f65080>,
 'mlp_num_layers': <hyperopt.pyll.base.Apply at 0x136f65160>,
 'mlp_num_fan_out': <hyperopt.pyll.base.Apply at 0x136ede080>}

In a `DenseBaseline` model, only `mlp_num_units`, `mlp_num_layers`, and `mlp_num_fan_out` have pre-defined hyper-space. In other words, only these hyper-parameters will change values during a tuning. Other hyper-parameters, like `mlp_activation_func`, are fixed and will not change.

In [16]:
def sample_and_build(params):
    sample = mz.hyper_spaces.sample(params.hyper_space)
    print('if sampled:', sample, '\n')
    params.update(sample)
    print('the built model will have:\n')
    print(params, '\n\n\n')

for _ in range(3):
    sample_and_build(model.params)

if sampled: {'mlp_num_fan_out': 28.0, 'mlp_num_layers': 3.0, 'mlp_num_units': 471.0} 

the built model will have:

model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 471
mlp_num_layers                3
mlp_num_fan_out               28
mlp_activation_func           relu 



if sampled: {'mlp_num_fan_out': 40.0, 'mlp_num_layers': 2.0, 'mlp_num_units': 25.0} 

the built model will have:

model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 25
mlp_num_layers                2
mlp_num_fan_out               40
mlp_activation_func           relu 



if sampled

In [ ]:
This is similar to the process of a tuner sampling model hyper-parameters, but with one key difference: a tuner's hyper-space is **suggestive**. This means the sampling process in a tuner is not truely random but skewed. Scores of the past samples affect future choices: a tuner with more runs knows better about its hyper-space, and take samples in a way that will likely yields better scores.

For more details, consult tuner's backend: [hyperopt](http://hyperopt.github.io/hyperopt/), and the search algorithm tuner uses: [Tree of Parzen Estimators (TPE)](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)

Hyper-spaces can also be represented in a human-readable format.

In [8]:
print(model.params.get('mlp_num_units').hyper_space)

quantitative uniform distribution in  [16, 512), with a step size of 1


In [9]:
model.params.to_frame()[['Name', 'Hyper-Space']]

,Name,Hyper-Space
0,model_class,None
1,input_shapes,None
2,task,None
3,optimizer,None
4,with_multi_layer_perceptron,None
5,mlp_num_units,"quantitative uniform distribution in [16, 512..."
6,mlp_num_layers,"quantitative uniform distribution in [1, 5), ..."
7,mlp_num_fan_out,"quantitative uniform distribution in [4, 128)..."
8,mlp_activation_func,None


### setting hyper-space

What if I want the tuner to choose `optimizer` among `adam`, `adagrad`, and `rmsprop`?

In [10]:
model.params.get('optimizer').hyper_space = mz.hyper_spaces.choice(['adam', 'adagrad', 'rmsprop'])

In [11]:
for _ in range(10):
    print(mz.hyper_spaces.sample(model.params.hyper_space))

{'mlp_num_fan_out': 20.0, 'mlp_num_layers': 4.0, 'mlp_num_units': 42.0, 'optimizer': 'adam'}
{'mlp_num_fan_out': 96.0, 'mlp_num_layers': 3.0, 'mlp_num_units': 422.0, 'optimizer': 'adagrad'}
{'mlp_num_fan_out': 124.0, 'mlp_num_layers': 4.0, 'mlp_num_units': 65.0, 'optimizer': 'adagrad'}
{'mlp_num_fan_out': 36.0, 'mlp_num_layers': 3.0, 'mlp_num_units': 176.0, 'optimizer': 'adagrad'}
{'mlp_num_fan_out': 128.0, 'mlp_num_layers': 4.0, 'mlp_num_units': 448.0, 'optimizer': 'adam'}
{'mlp_num_fan_out': 36.0, 'mlp_num_layers': 3.0, 'mlp_num_units': 371.0, 'optimizer': 'rmsprop'}
{'mlp_num_fan_out': 8.0, 'mlp_num_layers': 5.0, 'mlp_num_units': 486.0, 'optimizer': 'adam'}
{'mlp_num_fan_out': 24.0, 'mlp_num_layers': 4.0, 'mlp_num_units': 158.0, 'optimizer': 'adam'}
{'mlp_num_fan_out': 56.0, 'mlp_num_layers': 5.0, 'mlp_num_units': 199.0, 'optimizer': 'adagrad'}
{'mlp_num_fan_out': 100.0, 'mlp_num_layers': 5.0, 'mlp_num_units': 437.0, 'optimizer': 'adagrad'}


What about setting `mlp_num_layers` to a fixed value of 2?

In [12]:
model.params['mlp_num_layers'] = 2
model.params.get('mlp_num_layers').hyper_space = None

In [13]:
for _ in range(10):
    print(mz.hyper_spaces.sample(model.params.hyper_space))

{'mlp_num_fan_out': 100.0, 'mlp_num_units': 254.0, 'optimizer': 'rmsprop'}
{'mlp_num_fan_out': 60.0, 'mlp_num_units': 361.0, 'optimizer': 'adagrad'}
{'mlp_num_fan_out': 60.0, 'mlp_num_units': 495.0, 'optimizer': 'rmsprop'}
{'mlp_num_fan_out': 88.0, 'mlp_num_units': 494.0, 'optimizer': 'adam'}
{'mlp_num_fan_out': 24.0, 'mlp_num_units': 259.0, 'optimizer': 'adagrad'}
{'mlp_num_fan_out': 96.0, 'mlp_num_units': 422.0, 'optimizer': 'rmsprop'}
{'mlp_num_fan_out': 44.0, 'mlp_num_units': 321.0, 'optimizer': 'rmsprop'}
{'mlp_num_fan_out': 68.0, 'mlp_num_units': 276.0, 'optimizer': 'adagrad'}
{'mlp_num_fan_out': 108.0, 'mlp_num_units': 35.0, 'optimizer': 'adam'}
{'mlp_num_fan_out': 120.0, 'mlp_num_units': 200.0, 'optimizer': 'adam'}


### using callbacks

A `Tuner`'s `callbacks` is `[mz.auto.tuner.callbacks.LogResult()]` by default. This means in addition to returning results at the end of a `tune` call, the tuner also prints out result of each run during the `tune` call.

To save the model along the way, use `mz.auto.tuner.callbacks.SaveModel`.

In [14]:
tuner.num_runs = 2
tuner.callbacks.append(mz.auto.tuner.callbacks.SaveModel())
results = tuner.tune()

                each new call to `tune` starts fresh. In other words,
                hyperspaces are suggestive only within the same `tune` call.
Run #1
Score: 0.125
model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adagrad
with_multi_layer_perceptron   True
mlp_num_units                 153
mlp_num_layers                2
mlp_num_fan_out               20
mlp_activation_func           relu

Run #2
Score: 0.08333333333333333
model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     rmsprop
with_multi_layer_perceptron   True
mlp_num_units                 259
mlp_num_layers                2
mlp_num_fan_out               60
mlp_activation_func           relu



This will save all built models to your `mz.USER_TUNED_MODELS_DIR`, and can be loaded by:

In [15]:
best_model_id = results['best']['model_id']
mz.load_model(mz.USER_TUNED_MODELS_DIR.joinpath(best_model_id))

To load a pre-trained embedding layer into a built model during a tuning process, use `mz.auto.tuner.callbacks.LoadEmbeddingMatrix`.

In [16]:
glove_50 = mz.datasets.embeddings.load_glove_embedding(dimension=50)
preprocessor = mz.models.DUET.get_default_preprocessor()
train = preprocessor.fit_transform(train_raw, verbose=0)
dev = preprocessor.transform(dev_raw, verbose=0)
params = mz.models.DUET.get_default_params()
params['task'] = mz.tasks.Ranking()
params['input_shapes'] = preprocessor.context['input_shapes']
params['embedding_input_dim'] = preprocessor.context['vocab_size']
params['embedding_output_dim'] = 50

In [17]:
embedding_matrix = glove_50.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
load_embedding_matrix_callback = mz.auto.tuner.callbacks.LoadEmbeddingMatrix(embedding_matrix)

In [18]:
tuner = mz.auto.tuner.Tuner(
    params=params,
    train_data=train,
    test_data=dev,
    num_runs=1
)
tuner.callbacks.append(load_embedding_matrix_callback)
results = tuner.tune()

Run #1
Score: 0.16666666666666666
model_class                   <class 'matchzoo.models.duet.DUET'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_embedding                True
embedding_input_dim           285
embedding_output_dim          50
embedding_trainable           True
lm_filters                    32
lm_hidden_sizes               [32]
dm_filters                    32
dm_kernel_size                3
dm_q_hidden_size              32
dm_d_mpool                    3
dm_hidden_sizes               [32]
padding                       same
activation_func               relu
dropout_rate                  0.72



### make your own callbacks

To build your own callbacks, inherit `mz.auto.tuner.callbacks.Callback` and overrides corresponding methods.

A run proceeds in the following way:

- run start (callback)
- build model
- build end (callback)
- fit and evaluate model
- collect result
- run end (callback)

This process is repeated for `num_runs` times in a tuner.

For example, say I want to verify if my embedding matrix is correctly loaded.

In [19]:
import numpy as np

class ValidateEmbedding(mz.auto.tuner.callbacks.Callback):
    def __init__(self, embedding_matrix):
        self._matrix = embedding_matrix
        
    def on_build_end(self, tuner, model):
        loaded_matrix = model.get_embedding_layer().get_weights()[0]
        if np.isclose(self._matrix, loaded_matrix).all():
            print("Yes! The my embedding is correctly loaded!")

In [20]:
validate_embedding_matrix_callback = ValidateEmbedding(embedding_matrix)

In [21]:
tuner = mz.auto.tuner.Tuner(
    params=params,
    train_data=train,
    test_data=dev,
    num_runs=1,
    callbacks=[load_embedding_matrix_callback, validate_embedding_matrix_callback]
)
tuner.callbacks.append(load_embedding_matrix_callback)
results = tuner.tune()

Yes! The my embedding is correctly loaded!


## sandbox

In [1]:
import matchzoo as mz
train_raw = mz.datasets.toy.load_data('train', 'classification')
dev_raw = mz.datasets.toy.load_data('dev', 'classification')
test_raw = mz.datasets.toy.load_data('test', 'classification')
embedding = mz.datasets.embeddings.load_glove_embedding()
task = mz.tasks.Classification(num_classes=2)

Using TensorFlow backend.


In [ ]:
import matchzoo as mz
train_raw = mz.datasets.toy.load_data('train')
dev_raw = mz.datasets.toy.load_data('dev')
test_raw = mz.datasets.toy.load_data('test')
embedding = mz.datasets.embeddings.load_glove_embedding()
task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))

In [2]:
classes = mz.models.list_available()

In [20]:
train = prpr.transform(train_raw)
train_gen = gen_builder.build(train)
model.fit_generator(train_gen)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 13/13 [00:00<00:00, 6232.25it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 100/100 [00:00<00:00, 5167.12it/s]
Processing text_right with transform: 100%|██████████| 100/100 [00:00<00:00, 73623.03it/s]


Epoch 1/1
1/1 [==============================] - 0s 328ms/step - loss: 0.6688


In [98]:
bests = []
for model_class in classes[1:]:
    print(model_class)
    model, preprocessor, gen_builder, embedding_matrix = mz.prepare(
        task=task,
        model_class=model_class,
        data_pack=train_raw,
        embedding=embedding
    )
    train = preprocessor.transform(train_raw, verbose=0)
    test = preprocessor.transform(test_raw, verbose=0)
    train_gen = gen_builder.build(train)
    test_gen = gen_builder.build(test)
    tuner = mz.auto.Tuner(
        params=model.params,
        train_data=train_gen,
        test_data=test_gen,
        fit_kwargs=dict(epochs=1),
        num_runs=2
    )
    if 'with_embedding' in model.params:
        tuner.callbacks.append(mz.tuner.callbacks.LoadEmbeddingMatrix(embedding_matrix))
    results = tuner.tune()
    bests.append(results['best'])
print(bests)

<class 'matchzoo.models.dssm.DSSM'>
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 1.5347
Run #1
Score: 1.0
model_class                   <class 'matchzoo.models.dssm.DSSM'>
input_shapes                  [(1761,), (1761,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 80
mlp_num_layers                3
mlp_num_fan_out               52
mlp_activation_func           relu

Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 1.6049
Run #2
Score: 0.2
model_class                   <class 'matchzoo.models.dssm.DSSM'>
input_shapes                  [(1761,), (1761,)]
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 240
mlp_num_layers                3
mlp_num_fan_out               84
mlp_activation_func           relu

<class 'matchzoo.models.cdssm.CDSSM'>
Epoch 1/1
1/1 [

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 1.9335
Run #1
Score: 0.25
model_class                   <class 'matchzoo.models.knrm.KNRM'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_embedding                True
embedding_input_dim           285
embedding_output_dim          50
embedding_trainable           True
kernel_num                    14
sigma                         0.11
exact_sigma                   0.001

Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 8.3235
Run #2
Score: 0.2
model_class                   <class 'matchzoo.models.knrm.KNRM'>
input_shapes                  [(30,), (30,)]
task                          Ranking Task
optimizer                     adam
with_embedding                True
embedding_input_dim           285
embedding_output_dim          50
embedding_trainable           True
kernel_num                    9
sigma                   

TypeError: can only concatenate tuple (not "int") to tuple

In [8]:
import dill

In [21]:
dill.dump(train_gen, open('.tmpdill', 'wb'))

In [22]:
train_load = dill.load(open('.tmpdill', 'rb'))

In [27]:
train_load._data_pack.left

,text_left,length_left
id_left,,
Q1,"[23, 9, 223, 252, 73, 0, 0, 0, 0, 0, 0, 0, 0, ...",5
Q2,"[23, 9, 90, 220, 133, 90, 163, 232, 261, 262, ...",15
Q5,"[23, 37, 134, 143, 62, 0, 0, 0, 0, 0, 0, 0, 0,...",5
Q6,"[23, 136, 30, 90, 137, 7, 42, 2, 0, 0, 0, 0, 0...",8
Q7,"[23, 236, 103, 280, 118, 81, 210, 101, 0, 0, 0...",8
Q9,"[23, 236, 107, 101, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4
Q10,"[23, 224, 64, 178, 34, 101, 0, 0, 0, 0, 0, 0, ...",6
Q12,"[23, 15, 37, 141, 127, 28, 31, 82, 206, 265, 0...",10
Q13,"[23, 15, 30, 90, 18, 190, 279, 0, 0, 0, 0, 0, ...",7


In [8]:
# keeper = mz.ZooKeeper(mz.models.DSSM, task, embedding=embedding)
# model, preprocessor, gen_builder, embedding_matrix = keeper.prepare(train_raw)
# train = preprocessor.transform(train_raw, verbose=0)
# test = preprocessor.transform(test_raw, verbose=0)
# test_gen = gen_builder.build(test)